In [1]:
!pip -q install colorlog
!pip -q install keras_radam
!pip -q install keras_tqdm
#!pip install tensorflow==2.1.0-rc1
!pip install tensorflow==2.1.0 

!pip install tensorflow_probability==0.8.0

     |████████████████████████████████| 421.8MB 18kB/s 
     |████████████████████████████████| 450kB 46.9MB/s 
     |████████████████████████████████| 3.9MB 46.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5ffa0468f3aae94659151fd6d2749a94318fa9ae1021c266362c074fc4cac23b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: 

# Settings

In [0]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

In [0]:
#%cd /content/
#!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
#%cd /content/Generative_Models/

In [0]:
#os.chdir('/content/Generative_Models/')
print(os.getcwd())

# Dataset loading

In [0]:
dataset_name = 'dsprites'
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

In [0]:
intermediate_dim = 20
inputs_shape=(64, 64, 1) # image shape
batch_size = 32
latents_dim = 6

In [0]:
TRAIN_BUF = 600
TEST_BUF = 100

In [0]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)

In [0]:
from data.gt_load.datasets import load

In [0]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

In [0]:
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.gt_datasets'
#absolute = abspath(DATA_PATH)
print(DATA_PATH)

In [0]:
!ls /content/Generative_Models/data/.gt_datasets/dsprites

In [0]:
eval_dataset = load(dataset_name='dsprites_full',dataset_path=DATA_PATH)

In [0]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'].numpy().max())
    break

In [0]:
_instance_scale

In [0]:
import numpy as np
from collections.abc import Iterable

In [0]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [0]:
_outputs_shape

# Model's Layers definition

In [0]:
units=intermediate_dim
c=50
menc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=units*c*c, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, units)),
    tf.keras.layers.Conv2DTranspose(filters=units*3, kernel_size=3, strides=(4, 4), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [0]:
model_name = dataset_name+'GAVAE_Dense_reconst_ell'
experiments_dir='experiments'+sep_local+model_name

In [0]:
from training.adversarial_basic.generative_adversarial.autoencoders.VAAE import VAAE as AE

In [0]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist

In [0]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [0]:
create_if_not_exist(_restore)
_restore

In [0]:
#to restore trained model, set filepath=_restore

In [0]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

In [0]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses
    

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'generative_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [0]:
#ae.compile(metrics=None)
discr2gen_rate = 0.001
gen2trad_rate = 0.0

ae.compile(
    adversarial_losses=generative_discriminator_losses,
    adversarial_weights={'generator_weight': gen2trad_rate, 'discriminator_weight': discr2gen_rate}
)

# Callbacks

In [0]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [0]:
gts_csv = os.path.join(experiments_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(experiments_dir, 'csv_dir', 'gtu_metrics')

In [0]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=10
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    gt_freq=10
)

In [0]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [0]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [0]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

In [0]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [0]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [0]:
import numpy as np

# Model Training

In [0]:
ae.fit(
    x=train_ds,
    input_kw='image',
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

# Model Evaluation

## inception_score

In [0]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [0]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [0]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [0]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [0]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [0]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [0]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [0]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [0]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [0]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_randomly

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [0]:
from training.generators.image_generation_testing import interpolate_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)